In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

In [ ]:
cols = ['patient_id', 'created_at', 'process_types.name', 'source.1', 'Operação',
       'date', 'status.Status.Name', 'zipCode', 'district',
       'Prémio comercial A', 'Seguradora', 'Seguro',
       'Prémio comercial B', 'Seguradora.B', 'Seguro.B',
       'Prémio comercial C', 'Seguradoras.C', 'Seguros.C',
       'dateOfBirth', 'date.analise', 'date.especialista', 'Date.neg']

seg_upd = pd.read_excel('/content/drive/MyDrive/Projects/Project Dr Finanças/SEG/WCS_Data_SEG24DEC21.xlsx', usecols = cols)

In [ ]:
seg_upd.head()

In [ ]:
seg_upd.info()

In [ ]:
seg_upd['process_types.name'].value_counts()

In [ ]:
seg_upd.describe()

In [ ]:
seg_upd['patient_id'].value_counts()

In [ ]:
print(seg_upd['Operação'].value_counts(dropna = False), '\n')

seg_upd['status.Status.Name'].value_counts(dropna = False)

2 - Get "patient_id" with only "status.Status.Name = Concluído : Com Success"

In [ ]:
seg_up_sucess = seg_upd[seg_upd['status.Status.Name'] == 'Concluído : Com Sucesso']
seg_up_sucess

In [ ]:
seg_up_sucess.info()

In [ ]:
seg_up_sucess['patient_id'].value_counts()

In [ ]:
seg_up_sucess.isna().sum()

2.1 - Confirm if there is values on Prémio Comercial A, B or C
that can mess with the mean all other values after the concat

In [ ]:
seg_up_NOT_sucess = seg_upd[seg_upd['status.Status.Name'] != 'Concluído : Com Sucesso']
seg_up_NOT_sucess

In [ ]:
seg_up_NOT_sucess.describe()

3 - Concatenation
- Creation of 3 Tables with different "Prémio comercial A, B and C" with respective "Seguro";
- Concat them all;
- Get average of insurance by patient_id and by Type of Insurance

Get info from the 3 tables

Get info for 'Prémio comercial A'

In [ ]:
seg_up_sucess.columns

In [ ]:
seg_up_sucess_A = seg_up_sucess.loc[:, ["patient_id", "Prémio comercial A", "Seguro"]]
seg_up_sucess_A.rename(columns = {'patient_id':'Patient_ID',
                                  'Prémio comercial A':'Premium Insurance',
                                  'Seguro':'Type of Insurance'}, inplace = True)
seg_up_sucess_A

In [ ]:
### remove NaN from 'Premium Insurance' ###

seg_up_sucess_A.dropna(subset = ['Premium Insurance'],  inplace = True)
seg_up_sucess_A

In [ ]:
seg_up_sucess_A['Patient_ID'].value_counts()

In [ ]:
seg_up_sucess_A_2 = seg_up_sucess_A.groupby(['Type of Insurance']).agg({'Patient_ID':'count',
                                                                        'Premium Insurance': 'sum'})
seg_up_sucess_A_2

Get info for 'Prémio comercial B'

In [ ]:
### data with only 'Concluído : Com Sucesso' and all NaN's ###
### row size 3806 ###
### getting 'Prémio comercial B' and others only.

# seg_up_sucess.head()

seg_up_sucess_B = seg_up_sucess.loc[:, ["patient_id", "Prémio comercial B", "Seguro.B"]]
seg_up_sucess_B.rename(columns = {'patient_id':'Patient_ID',
                                  'Prémio comercial B':'Premium Insurance',
                                  'Seguro.B':'Type of Insurance'}, inplace = True)
seg_up_sucess_B

### remove NaN from 'Premium Insurance' ###

seg_up_sucess_B.dropna(subset = ['Premium Insurance'],  inplace = True)
seg_up_sucess_B

In [ ]:
seg_up_sucess_B.value_counts()

Get info for 'Prémio comercial C'

In [ ]:
### data with only 'Concluído : Com Sucesso' and all NaN's ###
### row size 3806 ###
### getting 'Prémio comercial C' and others only.

# seg_up_sucess.head()

seg_up_sucess_C = seg_up_sucess.loc[:, ["patient_id", "Prémio comercial C", "Seguros.C"]]
seg_up_sucess_C.rename(columns = {'patient_id':'Patient_ID',
                                  'Prémio comercial C':'Premium Insurance',
                                  'Seguros.C':'Type of Insurance'}, inplace = True)
seg_up_sucess_C

### remove NaN from 'Premium Insurance' ###

seg_up_sucess_C.dropna(subset = ['Premium Insurance'],  inplace = True)
seg_up_sucess_C

In [ ]:
seg_up_sucess_C.value_counts()

Concatenate

In [ ]:
### concatenate all 3 tables and then will check for duplicates

seg_up_sucess_total = pd.concat([seg_up_sucess_A, seg_up_sucess_B, seg_up_sucess_C])
seg_up_sucess_total

In [ ]:
seg_up_sucess_total['Premium Insurance'].mean()

In [ ]:
seg_avg_pat = seg_up_sucess_total['Patient_ID'].value_counts().to_frame()
seg_avg_pat

Download table to get Avg Insurance by Patient

In [ ]:
### DOWNLOAD the table above to get Avg Insurances by Patient ID ###

# from openpyxl import load_workbook

# seg_avg_pat.to_excel(r'/content/drive/MyDrive/Projects/Project Dr Finanças/SEG/Seguros_Avg_by_Patient_Final.xlsx', index = True)

In [ ]:
########## Final table to answer Scope ##########

### groupby().unique() -> 3639 patient_id

seg_up_sucess_total_final = seg_up_sucess_total.groupby(['Type of Insurance'],
                                                        as_index = False).agg({'Patient_ID':'nunique',
                                                                               'Premium Insurance': 'sum'})
print('Total of Patients: ',sum(seg_up_sucess_total_final['Patient_ID']),'\n')
print('Total Amount of Premium Insurance: ',round(sum(seg_up_sucess_total_final['Premium Insurance']), 2),'\n')
seg_up_sucess_total_final

In [ ]:
#### create extra columns '% Pre_Ins" #####

seg_up_sucess_total_final['% Pre_Ins'] = seg_up_sucess_total_final['Premium Insurance']/sum(seg_up_sucess_total_final['Premium Insurance'])*100
# seg_up_sucess_total_final

#### create extra columns '% Patient_ID" #####

seg_up_sucess_total_final['% Patient_ID'] = seg_up_sucess_total_final['Patient_ID']/sum(seg_up_sucess_total_final['Patient_ID'])*100
seg_up_sucess_total_final

In [ ]:
### data visualization from this df seg_up_sucess_total_final ###

seg_up_sucess_total_final.groupby(by = ['Type of Insurance']).mean().plot(kind = 'pie', 
                                                   y = '% Pre_Ins',
                                                   figsize = (15, 10),
                                                   autopct = '%1.1f%%')

4 - How many days take between each step

Date Created until date.Analise

In [ ]:
seg_up_sucess.columns

In [ ]:
seg_up_sucess.info()

In [ ]:
seg_up_sucess_dates = seg_up_sucess.loc[:, ["patient_id", 'date.analise', 
                                            'date.especialista',
                                            'Date.neg']]
# seg_up_sucess_A.rename(columns = {'patient_id':'Patient_ID',
#                                   'Prémio comercial A':'Premium Insurance',
#                                   'Seguro':'Type of Insurance'}, inplace = True)

print(seg_up_sucess_dates.info())
seg_up_sucess_dates

In [ ]:
len(seg_up_sucess_dates.patient_id.value_counts())

In [ ]:
##### STEP 1 DAYS ######
#### date.analise -> date.especialista without fillna() and 3806 rows #### 
## below i tried to just use the date.epsecialista with values and the result is the same as this one, so NaN works as 0 ####

# seg_up_sucess_dates_with_nan = seg_up_sucess_dates[seg_up_sucess_dates['date.especialista'] != 0]

# seg_up_sucess_dates_with_nan

seg_up_sucess_dates["Days from date.analise to date.especialista"] = seg_up_sucess_dates['date.especialista'] - seg_up_sucess_dates['date.analise']
seg_up_sucess_dates["Days from date.analise to date.especialista"] = seg_up_sucess_dates["Days from date.analise to date.especialista"].astype('timedelta64[D]')
print('Days from date.analise to date.especialista - MIN:',seg_up_sucess_dates["Days from date.analise to date.especialista"].min())
print('Days from date.analise to date.especialista - MAX:',seg_up_sucess_dates["Days from date.analise to date.especialista"].max())
print('Days from date.analise to date.especialista - MEAN:',seg_up_sucess_dates["Days from date.analise to date.especialista"].mean())
print(len(seg_up_sucess_dates["Days from date.analise to date.especialista"]))
print('Days from date.analise to date.especialista - MEAN Wihtout Negatives (-1):',(seg_up_sucess_dates["Days from date.analise to date.especialista"] >=0).mean())
# print('Len of the column',len(seg_up_sucess_dates["Days from date.analise to date.especialista"] >=0))

######## this was just to understand the mean of the positive days #######
only_positive_values_days_step1 = seg_up_sucess_dates[seg_up_sucess_dates["Days from date.analise to date.especialista"] >=0]
print('\n',only_positive_values_days_step1['Days from date.analise to date.especialista'].mean())
print(len(only_positive_values_days_step1['Days from date.analise to date.especialista']))
############################################################################


## plot just to have an idea of the mean and outliers
sns.boxplot(x = seg_up_sucess_dates["Days from date.analise to date.especialista"])

# seg_up_sucess_dates

In [ ]:
#### date.especialista --> Date.neg without fillna() and 3806 rows ####

seg_up_sucess_dates["Days from date.especialista to Date.neg"] = seg_up_sucess_dates['Date.neg'] - seg_up_sucess_dates['date.especialista']
seg_up_sucess_dates["Days from date.especialista to Date.neg"] = seg_up_sucess_dates["Days from date.especialista to Date.neg"].astype('timedelta64[D]')
print('Days from date.especialista to Date.neg - MIN:',seg_up_sucess_dates["Days from date.especialista to Date.neg"].min())
print('Days from date.especialista to Date.neg - MAX:',seg_up_sucess_dates["Days from date.especialista to Date.neg"].max())
print('Days from date.especialista to Date.neg - MEAN:',seg_up_sucess_dates["Days from date.especialista to Date.neg"].mean())
print(len(seg_up_sucess_dates['Days from date.especialista to Date.neg']))
print('Days from date.especialista to Date.neg - MEAN Wihtout Negatives (-1):',(seg_up_sucess_dates["Days from date.especialista to Date.neg"] >=0).mean())


######## this was just to understand the mean of the positive days #######
only_positive_values_days_step2 = seg_up_sucess_dates[seg_up_sucess_dates["Days from date.especialista to Date.neg"] >=0]
print('\n',only_positive_values_days_step2['Days from date.especialista to Date.neg'].mean())
print(len(only_positive_values_days_step2['Days from date.especialista to Date.neg']))
# print('Len of the column',len(seg_up_sucess_dates["Days from date.especialista to Date.neg"] >=0))
############################################################################

## plot just to have an idea of the mean and outliers
sns.boxplot(x = seg_up_sucess_dates["Days from date.especialista to Date.neg"])

# seg_up_sucess_dates

In [ ]:
#### date.analise --> Date.neg ##### FULL PROCESS

seg_up_sucess_dates["Days from date.analise to Date.neg"] = seg_up_sucess_dates['Date.neg'] - seg_up_sucess_dates['date.analise']
seg_up_sucess_dates["Days from date.analise to Date.neg"] = seg_up_sucess_dates["Days from date.analise to Date.neg"].astype('timedelta64[D]')
print('Days from date.analise to Date.neg - MIN:',seg_up_sucess_dates["Days from date.analise to Date.neg"].min())
print('Days from date.analise to Date.neg - MAX:',seg_up_sucess_dates["Days from date.analise to Date.neg"].max())
print('Days from date.analise to Date.neg - MEAN:',seg_up_sucess_dates["Days from date.analise to Date.neg"].mean())
print(len(seg_up_sucess_dates["Days from date.especialista to Date.neg"]))
print('Days from date.analise to Date.neg - MEAN Wihtout Negatives (-1):',(seg_up_sucess_dates["Days from date.especialista to Date.neg"] >=0).mean())


######## this was just to understand the mean of the positive days #######
only_positive_values_days_full_process = seg_up_sucess_dates[seg_up_sucess_dates["Days from date.analise to Date.neg"] >=0]
print('\n',only_positive_values_days_full_process['Days from date.analise to Date.neg'].mean())
print(len(only_positive_values_days_full_process['Days from date.analise to Date.neg']))
# print('Len of the column',len(seg_up_sucess_dates["Days from date.analise to Date.neg"] >=0))
############################################################################

## plot just to have an idea of the mean and outliers
sns.boxplot(x = seg_up_sucess_dates["Days from date.analise to Date.neg"])

seg_up_sucess_dates

Download table to use as steps

In [ ]:
### download of table dates to use in Tableau ###

# from openpyxl import load_workbook

# seg_up_sucess_dates.to_excel(r'/content/drive/MyDrive/Projects/Project Dr Finanças/SEG/Seguros_Dates_Final.xlsx', index = False)

Download table for the Retention Rate

In [ ]:
### download of table for the retention rate to use in Tableau ###

# from openpyxl import load_workbook

# seg_up_sucess.to_excel(r'/content/drive/MyDrive/Projects/Project Dr Finanças/SEG/Seguros_Retention_Rate.xlsx', index = False)

5 - RETENTION RATE

Year 2021

In [ ]:
# get all days from 2020 so needs to be < "01-01-2021" and not <= "31-12-2020" (because doesn't get all days)

present_customers_until_end_2020 = seg_up_sucess[seg_up_sucess.created_at < "01-01-2021"] 

print('MIN:', present_customers_until_end_2020.created_at.min()) #confirmation of min() date
print('MAX:', present_customers_until_end_2020.created_at.max()) #confirmation of max() date

start_of_year_clients = list(present_customers_until_end_2020.patient_id.unique())
# start_of_year_clients
total_start_year_clients = len(start_of_year_clients)
total_start_year_clients

In [ ]:
transactions_in_2021 = seg_up_sucess[seg_up_sucess.created_at > "01-01-2021"] # 01 January is bank holiday don't forget ;)

print('MIN date of 2021:', transactions_in_2021.created_at.min()) #confirmation of min() date
print('MAX date of 2021:', transactions_in_2021.created_at.max()) #confirmation of max() date

customers_transactions_in_2021 = list(transactions_in_2021.patient_id.unique())
len(customers_transactions_in_2021)

In [ ]:
retention_customers = 0
for client in start_of_year_clients:
  if client in customers_transactions_in_2021:
    retention_customers +=1
retention_customers

In [ ]:
retention_rate = round((retention_customers / total_start_year_clients ) * 100, 2)
retention_rate

Year 2020

In [ ]:
present_customers_until_end_2019 = seg_up_sucess[seg_up_sucess.created_at < "01-01-2020"]

print('MIN:', present_customers_until_end_2019.created_at.min()) #confirmation of min() date
print('MAX:', present_customers_until_end_2019.created_at.max()) #confirmation of max() date
start_of_year_clients = list(present_customers_until_end_2019.patient_id.unique())
total_start_year_clients = len(start_of_year_clients)
total_start_year_clients

In [ ]:
transactions_in_2020 = seg_up_sucess[(seg_up_sucess.created_at > "01-01-2020") &
                                  (seg_up_sucess.created_at < "01-01-2021")]

print('MIN date of 2020:', transactions_in_2020.created_at.min()) #confirmation of min() date
print('MAX date of 2020:', transactions_in_2020.created_at.max()) #confirmation of max() date
customers_transactions_in_2020 = list(transactions_in_2020.patient_id.unique())
len(customers_transactions_in_2020)

In [ ]:
retention_customers = 0
for client in start_of_year_clients:
  if client in customers_transactions_in_2020:
    retention_customers +=1
retention_customers

In [ ]:
retention_rate = round((retention_customers / total_start_year_clients ) * 100, 2)
retention_rate

Year 2019

In [ ]:
present_customers_until_end_2018 = seg_up_sucess[seg_up_sucess.created_at < "01-01-2019"]
print('MIN:', present_customers_until_end_2018.created_at.min()) #confirmation of min() date
print('MAX:', present_customers_until_end_2018.created_at.max()) #confirmation of max() date


start_of_year_clients = list(present_customers_until_end_2018.patient_id.unique())
total_start_year_clients = len(start_of_year_clients)
total_start_year_clients

In [ ]:
transactions_in_2019 = seg_up_sucess[(seg_up_sucess.created_at > "01-01-2019") &
                                  (seg_up_sucess.created_at < "01-01-2020")]

print('MIN date of 2019:', transactions_in_2019.created_at.min()) #confirmation of min() date
print('MAX date of 2019:', transactions_in_2019.created_at.max()) #confirmation of max() date
customers_transactions_in_2019 = list(transactions_in_2019.patient_id.unique())
len(customers_transactions_in_2019)

In [ ]:
retention_customers = 0
for client in start_of_year_clients:
  if client in customers_transactions_in_2019:
    retention_customers +=1
retention_customers

In [ ]:
retention_rate = round((retention_customers / total_start_year_clients ) * 100, 2)
retention_rate

6 - Same table from step 2 BUT with Seguradoras

In [ ]:
seg_up_sucess.columns

In [ ]:
seg_up_sucess_seguradora_A = seg_up_sucess.loc[:, ["patient_id", "Prémio comercial A", "Seguro", 'Seguradora']]
seg_up_sucess_seguradora_A.rename(columns = {'patient_id':'Patient_ID',
                                  'Prémio comercial A':'Premium Insurance',
                                  'Seguro':'Type of Insurance'}, inplace = True)
seg_up_sucess_seguradora_A

### remove NaN from 'Premium Insurance' ###

seg_up_sucess_seguradora_A.dropna(subset = ['Premium Insurance'],  inplace = True)
seg_up_sucess_seguradora_A

In [ ]:
seg_up_sucess_seguradora_B = seg_up_sucess.loc[:, ["patient_id", "Prémio comercial B", "Seguro.B", 'Seguradora.B']]
seg_up_sucess_seguradora_B.rename(columns = {'patient_id':'Patient_ID',
                                  'Prémio comercial B':'Premium Insurance',
                                  'Seguro.B':'Type of Insurance', 
                                  'Seguradora.B':'Seguradora'}, inplace = True)
seg_up_sucess_seguradora_B.info()

### remove NaN from 'Premium Insurance' ###

seg_up_sucess_seguradora_B.dropna(subset = ['Premium Insurance'],  inplace = True)
seg_up_sucess_seguradora_B

In [ ]:
# seg_up_sucess.head()

seg_up_sucess_seguradora_C = seg_up_sucess.loc[:, ["patient_id", "Prémio comercial C", "Seguros.C", 'Seguradoras.C']]
seg_up_sucess_seguradora_C.rename(columns = {'patient_id':'Patient_ID',
                                  'Prémio comercial C':'Premium Insurance',
                                  'Seguros.C':'Type of Insurance',
                                  'Seguradoras.C':'Seguradora'}, inplace = True)
seg_up_sucess_seguradora_C.info()

### remove NaN from 'Premium Insurance' ###

seg_up_sucess_seguradora_C.dropna(subset = ['Premium Insurance'],  inplace = True)
seg_up_sucess_seguradora_C

In [ ]:
### concatenate all 3 tables and then will check for duplicates

seg_up_sucess_total_seguradoras = pd.concat([seg_up_sucess_seguradora_A, seg_up_sucess_seguradora_B, seg_up_sucess_seguradora_C])
print(seg_up_sucess_total_seguradoras['Type of Insurance'].value_counts(),'\n')
seg_up_sucess_total_seguradoras

In [ ]:
seg_up_sucess_total_seguradoras['Patient_ID'].value_counts()

Download of table to see type of insurance and name of insurance

In [ ]:
### save file for Excel ####

# from openpyxl import load_workbook

# seg_up_sucess_total_seguradoras.to_excel(r'/content/drive/MyDrive/Projects/Project Dr Finanças/SEG\Seguros_Final.xlsx', index = False)

In [ ]:
########## Final table to answer Scope ##########

### groupby().unique() -> 3639 patient_id

seg_up_sucess_total_seguradoras_final = seg_up_sucess_total_seguradoras.groupby(['Type of Insurance', 'Seguradora'],
                                                        as_index = False).agg({'Patient_ID':'nunique',
                                                                               'Premium Insurance': 'sum'})
print('Total of Patients: ',sum(seg_up_sucess_total_seguradoras_final['Patient_ID']),'\n')
print('Total Amount of Premium Insurance: ',round(sum(seg_up_sucess_total_seguradoras_final['Premium Insurance']), 2),'\n')
seg_up_sucess_total_seguradoras_final

In [ ]:
fig, ax = plt.subplots(figsize= (10,8))

ax = sns.barplot(x="Type of Insurance",
                 y="Patient_ID",
                 hue="Seguradora",
                 data=seg_up_sucess_total_seguradoras_final)

ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

ax

In [ ]:
seg_up_sucess_total_seguradoras_final.Seguradora.values

In [ ]:
# fig, ax = plt.subplots(figsize= (20,15))

from matplotlib import rcParams

rcParams['figure.figsize'] = 12,8

g = sns.catplot(x="Seguradora", y="Patient_ID", col="Type of Insurance",
                data=seg_up_sucess_total_seguradoras_final,
            kind="bar", col_wrap = 4)
(g.set_axis_labels("", "Total Patient")
  .set_xticklabels(["Allianz", "Fidelidade", "Liberty",
                    'Mafre', 'Tranquilidade', 'Zurich',
                    'Ageas', 'Vitoria', 'Real Vida', 'Saude Prime',
                    'April', 'Metlife', 'Prevoir'], rotation = 90)
  .set_titles("{col_name}")
  .despine(left=True)) 

plt.show()

7 - Customer Lifetime Value

In [ ]:
seg_up_sucess_total.info()

In [ ]:
seg_up_sucess_total.describe()

In [ ]:
seg_up_sucess_total['Patient_ID'].value_counts()

In [ ]:
seg_up_sucess_total['Premium Insurance'].value_counts()

In [ ]:
seg_up_sucess_total

In [ ]:
seg_up_sucess_total_CLV = seg_up_sucess_total.groupby(['Patient_ID']).agg({'Premium Insurance':'sum'}).reset_index()
seg_up_sucess_total_CLV

Set the CLV Scores

In [ ]:
clv_seg_1 = seg_up_sucess_total_CLV[seg_up_sucess_total_CLV['Premium Insurance'] < 75]

print(clv_seg_1.count())
print('Min Score 1:', clv_seg_1['Premium Insurance'].min())
print('Max Score 1:', clv_seg_1['Premium Insurance'].max())
print('Avg Score 1:', clv_seg_1['Premium Insurance'].mean())

In [ ]:
clv_seg_2 = seg_up_sucess_total_CLV[(seg_up_sucess_total_CLV['Premium Insurance'] > 75) &
                           (seg_up_sucess_total_CLV['Premium Insurance'] < 150)]  # before between 125 and 1250 - 2944pid 

print(clv_seg_2.count())
print('Min Score 2:', clv_seg_2['Premium Insurance'].min())
print('Max Score 2:', clv_seg_2['Premium Insurance'].max())
print('Avg Score 2:', clv_seg_2['Premium Insurance'].mean())

In [ ]:
clv_seg_3 = seg_up_sucess_total_CLV[(seg_up_sucess_total_CLV['Premium Insurance'] > 150) &
                           (seg_up_sucess_total_CLV['Premium Insurance'] < 500)]  # before between 1250 and 2500 - 72pid 

print(clv_seg_3.count())
print('Min Score 3:', clv_seg_3['Premium Insurance'].min())
print('Max Score 3:', clv_seg_3['Premium Insurance'].max())
print('Avg Score 3:', clv_seg_3['Premium Insurance'].mean())

In [ ]:
clv_seg_4 = seg_up_sucess_total_CLV[(seg_up_sucess_total_CLV['Premium Insurance'] > 500) &
                           (seg_up_sucess_total_CLV['Premium Insurance'] < 1000)]

print(clv_seg_4.count())
print('Min Score 4:', clv_seg_4['Premium Insurance'].min())
print('Max Score 4:', clv_seg_4['Premium Insurance'].max())
print('Avg Score 4:', clv_seg_4['Premium Insurance'].mean())

In [ ]:
clv_seg_5 = seg_up_sucess_total_CLV[(seg_up_sucess_total_CLV['Premium Insurance'] > 1000)]
                                
print(clv_seg_5.count())
print('Min Score 5:', clv_seg_5['Premium Insurance'].min())
print('Max Score 5:', clv_seg_5['Premium Insurance'].max())
print('Avg Score 5:', clv_seg_5['Premium Insurance'].mean())

Def function to add scores on table

In [ ]:
def score_clv(x):
  if x < 75:
    return 1
  elif x >= 75 and x < 150:
    return 2
  elif x >= 150 and x < 500:
    return 3
  elif x >= 500 and x < 1000:
    return 4
  else:
    return 5

seg_up_sucess_total_CLV['CLV Score'] = seg_up_sucess_total_CLV['Premium Insurance'].apply(score_clv)

In [ ]:
seg_up_sucess_total_CLV

In [ ]:
seg_up_sucess_total_CLV['CLV Score'].value_counts()

Download table with CLV scores to use on Tableau

In [ ]:
### save file for Excel ####

# from openpyxl import load_workbook

# seg_up_sucess_total_CLV.to_excel(r'/content/drive/MyDrive/Projects/Project Dr Finanças/SEG/Seguros_Final_CLV_Scores.xlsx', index = False)